# semantic segmentation

In [4]:
from ultralytics import YOLO
import yaml
import os
import shutil
from datetime import datetime


def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def load_model(model_path='yolov8n-seg.pt'):
    return YOLO(model_path)


def define_paths(base_path):
    paths = {
        'base': base_path,
        'train_images': os.path.join(base_path, 'images/train'),
        'val_images': os.path.join(base_path, 'images/val'),
        'test_images': os.path.join(base_path, 'images/test'),
        'train_labels': os.path.join(base_path, 'labels/train'),
        'val_labels': os.path.join(base_path, 'labels/val'),
        'test_labels': os.path.join(base_path, 'labels/test')
    }
    return paths


def check_paths(paths):
    for key, path in paths.items():
        if 'test' not in key:  # Test paths are optional
            assert os.path.exists(path), f"{key} path not found: {path}"


def create_dataset_dict(paths, nc, names):
    dataset = {
        'path': paths['base'],
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test' if os.path.exists(paths['test_images']) else '',
        'names': names,
        'nc': nc
    }
    return dataset


def save_dataset_yaml(dataset, filename='temp_data.yaml'):
    with open(filename, 'w') as outfile:
        yaml.dump(dataset, outfile, default_flow_style=False)
    return filename


def train_model(model, data_yaml, save_dir):
    model.train(data=data_yaml, epochs=100, imgsz=640, device='cuda', task='segment', save_dir=save_dir)


def evaluate_model(model, data_yaml, test_images, test_labels):
    if os.path.exists(test_images) and os.path.exists(test_labels):
        results = model.val(data=data_yaml, split='test')
        print("Test evaluation results:", results)


def move_best_model(runs_dir, target_dir):
    best_model_path = os.path.join(runs_dir, 'weights/best.pt')
    target_model_path = os.path.join(target_dir, 'best_model.pt')
    if os.path.exists(best_model_path):
        shutil.move(best_model_path, target_model_path)
        print(f"Best model saved at: {target_model_path}")
    return target_model_path


def main():
    dataset_base_path = '../obj_det_@_tracking/factory_dataset/yolo_dataset/dataset_semantic'
    models_base_dir = '../obj_det_@_tracking/trained_models'
    nc = 2  # number of classes
    names = ["folding_machine", "worker"]  # list of class names

    model = load_model()
    paths = define_paths(dataset_base_path)
    check_paths(paths)

    dataset = create_dataset_dict(paths, nc, names)
    data_yaml = save_dataset_yaml(dataset)

    # Create a new directory for this training run under trained_models
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    model_save_dir = os.path.join(models_base_dir, timestamp)
    ensure_dir(model_save_dir)

    try:
        # Train the model and save results in the runs folder within the new timestamp folder
        runs_dir = os.path.join(model_save_dir, 'runs')
        ensure_dir(runs_dir)
        model.train(data=data_yaml, epochs=100, imgsz=640, device='cpu', task='segment', save_dir=runs_dir)

        # Evaluate the model on the test dataset
        evaluate_model(model, data_yaml, paths['test_images'], paths['test_labels'])

        # Move the best model from runs to the model_save_dir
        move_best_model(runs_dir, model_save_dir)
    finally:
        # Clean up by removing the temporary dataset YAML file
        if os.path.exists(data_yaml):
            os.remove(data_yaml)


if __name__ == '__main__':
    main()


New https://pypi.org/project/ultralytics/8.2.57 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48  Python-3.11.0 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=temp_data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

train: Scanning C:\Users\MSI\Documents\GitHub\susu_summer24\obj_det_@_tracking\factory_dataset\yolo_dataset\dataset_semantic\labels\train... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<00:00, 247.03it/s]

train: New cache created: C:\Users\MSI\Documents\GitHub\susu_summer24\obj_det_@_tracking\factory_dataset\yolo_dataset\dataset_semantic\labels\train.cache



val: Scanning C:\Users\MSI\Documents\GitHub\susu_summer24\obj_det_@_tracking\factory_dataset\yolo_dataset\dataset_semantic\labels\val... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<00:00, 864.62it/s]

val: New cache created: C:\Users\MSI\Documents\GitHub\susu_summer24\obj_det_@_tracking\factory_dataset\yolo_dataset\dataset_semantic\labels\val.cache


...

      9/100         0G     0.5631      1.258      2.135     0.9138         79        640: 100%|██████████| 1/1 [00:40<00:00, 40.58s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:06<00:00,  6.20s/it]

                   all         16         32     0.0071          1      0.995      0.852     0.0071          1      0.995      0.802



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/1 [00:32<?, ?it/s]


KeyboardInterrupt: 